# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:

# import dependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Experiment, Dataset
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.data.dataset_factory import TabularDatasetFactory
#from azureml.train.hyperdrive import uniform, choice
from azureml.core import ScriptRunConfig
from azureml.core import Environment
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil


## Initialize Workspace and Create an Azure ML experiment

In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

AZML
RG-AZML
eastus
a748a023-896b-4719-8f8e-3d1ba62d6e35


In [5]:
experiment_name = 'ideb_regression'

experiment=Experiment(ws, experiment_name)

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [21]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'a748a023-896b-4719-8f8e-3d1ba62d6e35'
resource_group = 'RG-AZML'
workspace_name = 'AZML'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='dataset_ideb2019')
dataset.to_pandas_dataframe()

,Sigla_UF,Cod_Municipio,Nome_Municipio,Cod_Escola,Nome_Escola,Column6,TAprov2019_Total,TAprov2019_1_serie,TAprov2019_2_serie,TAprov2019_3_serie,TAprov2019_4_serie,Indicador_Rendimento,SAEB2019_Matematica,SAEB2019_Lingua Portuguesa,SAEB2019_Nota Media Padronizada,IDEB_2019
0,BA,2914703,Itaberaba,29083524,EE - COLEGIO ESTADUAL LIBERDADE,Estadual,43.3,33.7,37.3,69.8,-,0.42,217.25,226.70,3.13,1.3
1,AM,1300060,Amaturá,13103431,ESC EST INDIGENA MANUEL JOAQUIM SALDANHA FILHO...,Estadual,67.3,59.1,53.5,92.5,-,0.65,195.40,188.21,2.25,1.5
2,PA,1506708,Santana do Araguaia,15171582,EEEM JORCELI SESTARI SILVA ANEXO IV,Estadual,57.1,52.4,50.0,72.2,-,0.57,202.24,211.64,2.70,1.5
3,PI,2211001,Teresina,22128441,U E NOSSA SENHORA DO PERPETUO SOCORRO,Estadual,53.8,25.0,58.3,97.5,-,0.45,239.39,237.57,3.61,1.6
4,PI,2211001,Teresina,22131159,CETI PEDRA MOLE,Estadual,41.7,30.6,31.1,78.7,-,0.39,259.57,260.92,4.24,1.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12074,ES,3205200,Vila Velha,32078498,IFES - CAMPUS VILA VELHA,Federal,95.4,94.8,97.7,94.1,-,0.96,394.30,363.46,7.67,7.3
12075,PE,2611606,Recife,26119730,ESCOLA DE APLICACAO DO RECIFE,Estadual,98.2,100.0,94.4,100.0,-,0.98,396.83,360.43,7.66,7.5
12076,SP,3506003,Bauru,35922948,COL TEC INDUSTRIAL PROF ISAAC PORTAL ROLDAN UNESP,Estadual,98.8,97.1,100.0,99.2,-,0.99,393.83,356.23,7.55,7.5
12077,PE,2611606,Recife,26124297,COLEGIO DE APLICACAO DO CE DA UFPE,Federal,98.9,96.7,100.0,100.0,-,0.99,402.96,362.42,7.77,7.7


In [22]:
dataset = dataset.register(workspace = ws,
                           name = 'dataset_ideb2019',
                           description='training and test dataset',
                           create_new_version=True)
dataset

{
  "source": [
    "('workspaceblobstore', 'UI/01-11-2021_071425_UTC/DATASET_IDEB_ensino_medio_escolas_2019.txt')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "0d33519d-9ba1-4a92-a3d1-b36ee1213258",
    "name": "dataset_ideb2019",
    "version": 2,
    "description": "training and test dataset",
    "workspace": "Workspace.create(name='AZML', subscription_id='a748a023-896b-4719-8f8e-3d1ba62d6e35', resource_group='RG-AZML')"
  }
}

## Creating Compute

In [24]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [29]:

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(
                        evaluation_interval=3,
                        slack_factor=0.1

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
    {
    '--C': uniform(0.1, 0.9),
    '--max_iter': choice(10, 50, 100)
    }
)


#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
                source_directory="./",
                entry_script="train.py",
                compute_target=aml_compute,

hyperdrive_run_config = HyperDriveConfig(
                    primary_metric_name='r2_score',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_total_runs=20,
                    max_concurrent_runs=4,
                    hyperparameter_sampling=param_sampling,
                    policy=early_termination_policy,
                    estimator=estimator

SyntaxError: invalid syntax (<ipython-input-29-f52b10f7150f>, line 7)

## Submit your experiment

In [ ]:
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
assert(hyperdrive_run.get_status() == "Completed")

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run.get_metrics()

In [ ]:
print(best_run.get_details()['runDefinition']['arguments'])

In [ ]:
print(best_run.get_file_names())

In [ ]:
#TODO: Save the best model
model = best_run.register_model(model_name='ideb-hyper', model_path='outputs/model.joblib')
print('Best model registered')


model.download(target_dir="outputs", exist_ok=True)
print('model downloaded')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service